<a href="https://colab.research.google.com/github/analyticsariel/projects/blob/master/How_to_Get_Zillow_Agent_Contact_Information.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# How to Get Zillow Agent Contact Information | Python Tutorial
<i>Python tutorial to get a list of real estate agent in a city</i>
<br>

## Overview
| Detail Tag            | Information                                                                                        |
|-----------------------|----------------------------------------------------------------------------------------------------|
| Originally Created By | Ariel Herrera arielherrera@analyticsariel.com |
| External References   | Rapid API |
| Input Datasets        | [Zillow.com API](https://rapidapi.com/apimaker/api/zillow-com1/) |
| Output Datasets       | CSV file |
| Input Data Source     | JSON file |
| Output Data Source    | Pandas DataFrame |

## History
| Date         | Developed By  | Reason                                                |
|--------------|---------------|-------------------------------------------------------|
| 7th August 2022 | Ariel Herrera | Create notebook. |

## Background 🤓
- **Overview**: This notebook retrieves real estate agents on Zillow using the [Zillow.com API](https://rapidapi.com/apimaker/api/zillow-com1/). This includes contact information, number of listings, and review score.
- **Purpose**: This notebook is a Python tutorial to query the [Zillow.com API](https://rapidapi.com/apimaker/api/zillow-com1/). Below are examples on how to extract listings data for a single city and multiple cities.

## Getting Started ✅
1. Copy this notebook -> File -> Save a Copy in Drive
2. Directions
  - Create an account with [RapidAPI.com](rapidapi.com)
  - Copy your [RapidAPI key](https://rapidapi.com/blog/api-glossary/api-key/)
  - Store your RapidAPI key into an `api_key.csv` file OR replace the `rapid_api_key` variable <string> with your own RapidAPI key
  - Run notebook -> Runtime -> Run all

## Useful Resources 📑
- [Google Colab Cheat Sheet](https://towardsdatascience.com/cheat-sheet-for-google-colab-63853778c093)
- [Zillow.com API](https://rapidapi.com/apimaker/api/zillow-com1/)

## Questions / Feedback❓
- [Contact AnalyticsAriel](https://www.analyticsariel.com/contact-me)

## <font color="blue">Install Packages</font>

## <font color="blue">Imports</font>

In [ ]:
from google.colab import drive, files # specific to Google Colab
from googlesearch import search # get zpid
import pandas as pd
import numpy as np
import requests
import json
import time
import io
import re
import plotly.express as px


# show all columns
pd.set_option('display.max_columns', None)

## <font color="blue">Functions</font>

In [ ]:
def get_single_agent_req(rapid_api_key, 
                         location,
                         page=1):
  url = "https://zillow-com1.p.rapidapi.com/findAgent"

  querystring = {"locationText":location,"page":str(page)}

  headers = {
    "X-RapidAPI-Key": rapid_api_key,
    "X-RapidAPI-Host": "zillow-com1.p.rapidapi.com"
  }

  return requests.request("GET", url, headers=headers, params=querystring)

In [ ]:
def get_agents(rapid_api_key,
               location, 
               all_pages=False,
               sleep_time=0.5):
  # get single agent req
  response = get_single_agent_req(rapid_api_key, location)
  response_list = [response]

  try:

    if all_pages == True:
      start_time = time.time()
      # iterate through each page
      print('Retrieved data for page: 1')

      last_page = response.json()['pageInformation']['lastPage']

      if last_page == 1:
        return response_list
      elif last_page == 2:
        response = get_single_agent_req(rapid_api_key, location, page=2)
        return response_list.append(response)
      else:
        for i in range(2, last_page + 1):
          time.sleep(sleep_time) # force to sleep (based on API restrictions)
          print('Requesting data for page:', str(i))
          response = get_single_agent_req(rapid_api_key, location, page=i)
          response_list.append(response)
        
        print("--- %s seconds ---" % (time.time() - start_time))
        return response_list
    else:
      return response_list

  except:
  
    return response_list

In [ ]:
def transform_listings_to_df(response_list):
  return pd.concat([pd.json_normalize(data=r.json()['agents']) for r in response_list])

## <font color="blue">Locals & Constants</font>

In [ ]:
########################################################################
# OPTIONAL                                                             #
# 1) Create a file called 'api_keys.csv' to store your Rapid API key   #
# 2) Read in the file                                                  #
# 3) Set the Rapid API key to the rapid_api_key variable               #
########################################################################

# mount drive
drive.mount('/content/drive', force_remount=False)

# data location
file_dir = '/content/drive/My Drive/Colab Data/input/' # optional

Mounted at /content/drive


In [ ]:
# read in api key file
df_api_keys = pd.read_csv(file_dir + 'api_keys.csv')

# get keys
rapid_api_key = df_api_keys.loc[df_api_keys['API'] =='rapid']['KEY'].iloc[0] # replace this with your own key

## <font color="blue">Data</font>

### <font color="green">Section #1 - Single API Requests</font> 🤙
This section will cover how to make API requests to the [Zillow.com API](https://rapidapi.com/apimaker/api/zillow-com1/). It demonstrates how to use the <i>findAgent</i> end point.

Step #1 - <b>Select a location</b> either neighborhood, city, or zip code. Get data for the first page.

In [ ]:
url = "https://zillow-com1.p.rapidapi.com/findAgent"

querystring = {"locationText":"tampa, fl", # location
               "page":"1"} # page number

headers = {
	"X-RapidAPI-Key": rapid_api_key,
	"X-RapidAPI-Host": "zillow-com1.p.rapidapi.com"
}

response = requests.request("GET", url, headers=headers, params=querystring)

print(response.text) # view response

{"agents":[{"profileLink":{"text":"The Duncan Duo","href":"\/profile\/TheDuncanDuo\/"},"index":0,"reviewExcerpt":{"excerpt":"From our first meeting we knew we were in good hands.","href":"\/profile\/TheDuncanDuo\/#reviews","reviewLink":{"text":"Review 06\/13\/2022","href":"\/profile\/TheDuncanDuo\/#reviews"}},"zuid":"X1-ZUz193idtwb8qx_a9fjw","contact":{"graphic":{"image":{"alt":"The Duncan Duo's profile photo","src":"https:\/\/photos.zillowstatic.com\/h_g\/ISznw4b1334hh51000000000.jpg"},"isCircle":true},"summary":{"profileLink":{"text":"The Duncan Duo","href":"\/profile\/TheDuncanDuo\/"},"encZuid":"X1-ZUz193idtwb8qx_a9fjw","reviewLink":{"text":"924 total reviews (302 local)","href":"\/profile\/TheDuncanDuo\/#reviews"},"isReviewable":true,"reviewStars":{"rating":4.9,"href":"\/profile\/TheDuncanDuo\/#reviews","stars":"500"},"phone":"(813) 514-8504","featured":false,"license":"Agent License #: 3426012"}},"href":"\/profile\/TheDuncanDuo\/","hasActivity":true,"map":{"label":"In Tampa:","bin

Step #2 - View response in JSON format

In [ ]:
# view response, here we see agent data is nested in the "agents" column
pd.json_normalize(data=response.json())

,agents,status,count,region.mapBounds,region.type,region.id,region.locationText,pageInformation.lastPage,pageInformation.pageSize,pageInformation.resultCount,pageInformation.firstPage,pageInformation.currentPage
0,"[{'profileLink': {'text': 'The Duncan Duo', 'h...",success,13342,"[-82.64882, 27.817487, -82.259741, 28.171478]",6,41176,Tampa FL,25,10,13342,1,1


Step #3 - View `agent` data in a DataFrame (table format)

In [ ]:
# transform data to pandas dataframe
df = pd.json_normalize(data=response.json()['agents'])
print('Num of rows:', len(df))
print('Num of cols:', len(df.columns))
df.head()

Num of rows: 10
Num of cols: 40


,index,zuid,href,hasActivity,profileLink.text,profileLink.href,reviewExcerpt.excerpt,reviewExcerpt.href,reviewExcerpt.reviewLink.text,reviewExcerpt.reviewLink.href,contact.graphic.image.alt,contact.graphic.image.src,contact.graphic.isCircle,contact.summary.profileLink.text,contact.summary.profileLink.href,contact.summary.encZuid,contact.summary.reviewLink.text,contact.summary.reviewLink.href,contact.summary.isReviewable,contact.summary.reviewStars.rating,contact.summary.reviewStars.href,contact.summary.reviewStars.stars,contact.summary.phone,contact.summary.featured,contact.summary.license,map.label,map.bingLogo.bgY,map.bingLogo.height,map.bingLogo.bgX,map.bingLogo.width,map.bingLogo.src,map.image.alt,map.image.width,map.image.src,map.image.height,map.stats.listings,map.stats.recentSales,map.stats.review,map.businessName,reviewExcerpt.noExcerpt
0,0,X1-ZUz193idtwb8qx_a9fjw,/profile/TheDuncanDuo/,True,The Duncan Duo,/profile/TheDuncanDuo/,From our first meeting we knew we were in good...,/profile/TheDuncanDuo/#reviews,Review 06/13/2022,/profile/TheDuncanDuo/#reviews,The Duncan Duo's profile photo,https://photos.zillowstatic.com/h_g/ISznw4b133...,True,The Duncan Duo,/profile/TheDuncanDuo/,X1-ZUz193idtwb8qx_a9fjw,924 total reviews (302 local),/profile/TheDuncanDuo/#reviews,True,4.9,/profile/TheDuncanDuo/#reviews,500,(813) 514-8504,False,Agent License #: 3426012,In Tampa:,-28,19,0,65,"data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAA...",Map of The Duncan Duo's listings,400,https://maps.googleapis.com/maps/api/staticmap...,160,18 Listings,No Recent Sales,302 Local Reviews,RE/MAX Dynamic,NaN
1,1,X1-ZUy3e7771zscg9_4hje0,/profile/robertwilsontampa/,True,Robert Wilson,/profile/robertwilsontampa/,"And it was also a good reminder to me, to get ...",/profile/robertwilsontampa/#reviews,Review 04/05/2017,/profile/robertwilsontampa/#reviews,Robert Wilson's profile photo,https://photos.zillowstatic.com/h_g/IS2f40viuo...,True,Robert Wilson,/profile/robertwilsontampa/,X1-ZUy3e7771zscg9_4hje0,4 total reviews (4 local),/profile/robertwilsontampa/#reviews,True,5.0,/profile/robertwilsontampa/#reviews,500,(813) 784-4199,False,,In Tampa:,-28,19,0,65,"data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAA...",Map of Robert Wilson's listings,400,https://maps.googleapis.com/maps/api/staticmap...,160,7 Listings,No Recent Sales,4 Local Reviews,Exit Bayshore Realty,NaN
2,2,X1-ZUyokqbhe84i6h_7x5fk,/profile/TheVikingTeam/,True,Lotte Rogers LLC dba The Viking Team,/profile/TheVikingTeam/,Lotte was a terrific resource and very knowled...,/profile/TheVikingTeam/#reviews,Review 12/12/2017,/profile/TheVikingTeam/#reviews,Lotte Rogers LLC dba The Viking Team's profile...,https://photos.zillowstatic.com/h_g/ISnut9my6w...,True,Lotte Rogers LLC dba The Viking Team,/profile/TheVikingTeam/,X1-ZUyokqbhe84i6h_7x5fk,5 total reviews (1 local),/profile/TheVikingTeam/#reviews,True,5.0,/profile/TheVikingTeam/#reviews,500,(813) 344-4024,False,Agent License #: 686642,In Tampa:,-28,19,0,65,"data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAA...",Map of Lotte Rogers LLC dba The Viking Team's ...,400,https://maps.googleapis.com/maps/api/staticmap...,160,6 Listings,No Recent Sales,1 Local Review,RE/MAX Alliance Group,NaN
3,3,X1-ZU11u0j3wv7l2x5_7sxq5,/profile/Jeramiahbustin2/,True,Jeramiah Bustin,/profile/Jeramiahbustin2/,"Each time arriving at Tampa, Jeremiah had the ...",/profile/Jeramiahbustin2/#reviews,Review 02/24/2020,/profile/Jeramiahbustin2/#reviews,Jeramiah Bustin's profile photo,https://photos.zillowstatic.com/h_g/ISfseiy0gu...,True,Jeramiah Bustin,/profile/Jeramiahbustin2/,X1-ZU11u0j3wv7l2x5_7sxq5,15 total reviews (10 local),/profile/Jeramiahbustin2/#reviews,True,5.0,/profile/Jeramiahbustin2/#reviews,500,(813) 857-2120,False,,In Tampa:,-28,19,0,65,"data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAA...",Map of Jeramiah Bustin's listings,400,https://maps.googleapis.com/maps/api/staticmap...,160,3 Listings,No Recent Sales,10 Local Reviews,Engel & Voelkers South Tampa,NaN
4,4,X1-ZUw968fkzxw361_1

### <font color="green">Section #2 - Get ALL Pages for a Single City</font> 🏙
This section will cover how to get all pages related to a location for the <i>findAgent</i> end point.

Step #1 - Use the <b>get_agents</b> function and set `all_pages` to True

In [ ]:
agent_single_city_resp_list = get_agents(rapid_api_key, location="tampa, fl", all_pages=True)

Retrieved data for page: 1
Requesting data for page: 2
Requesting data for page: 3
Requesting data for page: 4
Requesting data for page: 5
Requesting data for page: 6
Requesting data for page: 7
Requesting data for page: 8
Requesting data for page: 9
Requesting data for page: 10
Requesting data for page: 11
Requesting data for page: 12
Requesting data for page: 13
Requesting data for page: 14
Requesting data for page: 15
Requesting data for page: 16
Requesting data for page: 17
Requesting data for page: 18
Requesting data for page: 19
Requesting data for page: 20
Requesting data for page: 21
Requesting data for page: 22
Requesting data for page: 23
Requesting data for page: 24
Requesting data for page: 25
--- 54.17665719985962 seconds ---


Step #2 - Concatenate each DataFrame into a single table

In [ ]:
# select columns
cols_list = ['zuid', 'profileLink.text', 'profileLink.href', 'reviewExcerpt.excerpt', 
       'reviewExcerpt.reviewLink.text', 'contact.graphic.image.src',
       'contact.summary.reviewLink.text','contact.summary.reviewStars.rating',
       'contact.summary.phone','contact.summary.license', 'map.stats.listings', 
       'map.stats.review', 'map.businessName']

# transform list of responses into a single dataframe
df_agents_single_city = transform_listings_to_df(agent_single_city_resp_list)[cols_list]
print('Num of rows:', len(df_agents_single_city))
print('Num of columns:', len(df_agents_single_city.columns))
df_agents_single_city.head() # preview first 5 rows

Num of rows: 250
Num of columns: 13


,zuid,profileLink.text,profileLink.href,reviewExcerpt.excerpt,reviewExcerpt.reviewLink.text,contact.graphic.image.src,contact.summary.reviewLink.text,contact.summary.reviewStars.rating,contact.summary.phone,contact.summary.license,map.stats.listings,map.stats.review,map.businessName
0,X1-ZUz193idtwb8qx_a9fjw,The Duncan Duo,/profile/TheDuncanDuo/,From our first meeting we knew we were in good...,Review 06/13/2022,https://photos.zillowstatic.com/h_g/ISznw4b133...,924 total reviews (302 local),4.9,(813) 514-8504,Agent License #: 3426012,18 Listings,302 Local Reviews,RE/MAX Dynamic
1,X1-ZUy3e7771zscg9_4hje0,Robert Wilson,/profile/robertwilsontampa/,"And it was also a good reminder to me, to get ...",Review 04/05/2017,https://photos.zillowstatic.com/h_g/IS2f40viuo...,4 total reviews (4 local),5.0,(813) 784-4199,,7 Listings,4 Local Reviews,Exit Bayshore Realty
2,X1-ZUyokqbhe84i6h_7x5fk,Lotte Rogers LLC dba The Viking Team,/profile/TheVikingTeam/,Lotte was a terrific resource and very knowled...,Review 12/12/2017,https://photos.zillowstatic.com/h_g/ISnut9my6w...,5 total reviews (1 local),5.0,(813) 344-4024,Agent License #: 686642,6 Listings,1 Local Review,RE/MAX Alliance Group
3,X1-ZU11u0j3wv7l2x5_7sxq5,Jeramiah Bustin,/profile/Jeramiahbustin2/,"Each time arriving at Tampa, Jeremiah had the ...",Review 02/24/2020,https://photos.zillowstatic.com/h_g/ISfseiy0gu...,15 total reviews (10 local),5.0,(813) 857-2120,,3 Listings,10 Local Reviews,Engel & Voelkers South Tampa
4,X1-ZUw968fkzxw361_1r7ky,Bill Lewis,/profile/blewis7109/,NaN,NaN,https://photos.zillowstatic.com/h_g/ISfo3redpa...,NaN,0.0,(727) 510-6353,,3 Listings,No Reviews,Berkshire Hathaway HomeServices Florida Proper...


### <font color="green">Section #3 - Get data for Multiple Cities</font> 🌆
This section will cover how to get all pages related to multiple locations for the <i>findAgent</i> end point.

Step #1 - Create a list of cities to get agent data for. Use the <b>get_agents</b> function and set `all_pages` to True

In [ ]:
city_list = ["tampa, fl", "st petersburg, fl", "lakeland, fl", "sarasota, fl", "spring hill, fl"]
df_list = []

for c in city_list:
  # get resp
  _agent_resp_list = get_agents(rapid_api_key, location=c, all_pages=True)
  # transform list of responses into a single dataframe
  _df = transform_listings_to_df(_agent_resp_list)[cols_list]
  _df['location'] = c
  df_list.append(_df)

df_agents = pd.concat(df_list) # combine all dataframes

In [ ]:
# view number of rows per dataframe
df_agents.groupby(['location'])['zuid'].count()

location
lakeland, fl         250
sarasota, fl         250
spring hill, fl      250
st petersburg, fl    250
tampa, fl            250
Name: zuid, dtype: int64

Step #2 - Clean up data. Create features to view distribution of data.

In [ ]:
df_feat = df_agents.copy()
df_feat['last_review_dt'] = df_feat.apply(lambda x: 
  x['reviewExcerpt.reviewLink.text'] if pd.isna(x['reviewExcerpt.reviewLink.text']) == True else pd.to_datetime(x['reviewExcerpt.reviewLink.text'][-10:]), axis=1)
df_feat['last_review_yr'] = df_feat.apply(lambda x: 
  x['last_review_dt'] if pd.isna(x['last_review_dt']) == True else x['last_review_dt'].year, axis=1)
df_feat['num_reviews'] = df_feat.apply(lambda x: 
  x['contact.summary.reviewLink.text'] if pd.isna(x['contact.summary.reviewLink.text']) == True else int(x['contact.summary.reviewLink.text'].split(' ')[0]), axis=1)
df_feat['num_local_reviews'] = df_feat.apply(lambda x: 
  np.nan if x['map.stats.review'].split(' ')[0] == 'No' else int(x['map.stats.review'].split(' ')[0]), axis=1)
df_feat['review_flag'] = df_feat.apply(lambda x: 
  False if pd.isna(x['num_reviews']) == True else True, axis=1)
df_feat['num_listings'] = df_feat.apply(lambda x: 
  x['map.stats.listings'] if x['map.stats.listings'].split(' ')[0] == 'No' else int(x['map.stats.listings'].split(' ')[0]), axis=1)
df_feat['phone_num'] = df_feat.apply(lambda x: 
  x['contact.summary.phone'] if pd.isna(x['contact.summary.phone']) == True else re.sub('[^A-Za-z0-9]+', '', x['contact.summary.phone']), axis=1)
df_feat['agent_license'] = df_feat.apply(lambda x: 
  np.nan if x['contact.summary.license'] == '' else x['contact.summary.license'].split(' ')[-1], axis=1)
df_feat['first_name'] = df_feat.apply(lambda x: 
  x['profileLink.text'].split(' ')[0], axis=1)
df_feat['last_name'] = df_feat.apply(lambda x: 
  x['profileLink.text'].split(' ')[-1], axis=1)
df_feat.head(2)

,zuid,profileLink.text,profileLink.href,reviewExcerpt.excerpt,reviewExcerpt.reviewLink.text,contact.graphic.image.src,contact.summary.reviewLink.text,contact.summary.reviewStars.rating,contact.summary.phone,contact.summary.license,map.stats.listings,map.stats.review,map.businessName,location,last_review_dt,last_review_yr,num_reviews,num_local_reviews,review_flag,num_listings,phone_num,agent_license,first_name,last_name
0,X1-ZUz193idtwb8qx_a9fjw,The Duncan Duo,/profile/TheDuncanDuo/,From our first meeting we knew we were in good...,Review 06/13/2022,https://photos.zillowstatic.com/h_g/ISznw4b133...,924 total reviews (302 local),4.9,(813) 514-8504,Agent License #: 3426012,18 Listings,302 Local Reviews,RE/MAX Dynamic,"tampa, fl",2022-06-13,2022,924.0,302.0,True,18,8135148504,3426012,The,Duo
1,X1-ZUy3e7771zscg9_4hje0,Robert Wilson,/profile/robertwilsontampa/,"And it was also a good reminder to me, to get ...",Review 04/05/2017,https://photos.zillowstatic.com/h_g/IS2f40viuo...,4 total reviews (4 local),5.0,(813) 784-4199,,7 Listings,4 Local Reviews,Exit Bayshore Realty,"tampa, fl",2017-04-05,2017,4.0,4.0,True,7,8137844199,NaN,Robert,Wilson


Step #3 - Data visualization 😍

In [ ]:
_df_grp = df_feat.groupby(['review_flag'])['zuid'].count().reset_index()\
  .rename(columns={'zuid': 'cnt'})\
  .sort_values(by=['review_flag'], ascending=False)
_df_grp['prct'] = _df_grp['cnt'] / _df_grp['cnt'].sum()
_df_grp

,review_flag,cnt,prct
1,True,1164,0.9312
0,False,86,0.0688


In [ ]:
fig = px.histogram(df_feat.loc[df_feat['review_flag'] == True], x="num_reviews")
fig.show()

In [ ]:
fig = px.histogram(df_feat, x="num_listings")
fig.show()

Step #4 - Filter down on a subset of agents. Remove duplicates.

In [ ]:
df_filter = df_feat.loc[(df_feat['review_flag'] == True) &
                        (df_feat['last_review_yr'] >= 2021)]\
  .drop(columns=['location'])\
  .drop_duplicates()
df_filter

,zuid,profileLink.text,profileLink.href,reviewExcerpt.excerpt,reviewExcerpt.reviewLink.text,contact.graphic.image.src,contact.summary.reviewLink.text,contact.summary.reviewStars.rating,contact.summary.phone,contact.summary.license,map.stats.listings,map.stats.review,map.businessName,last_review_dt,last_review_yr,num_reviews,num_local_reviews,review_flag,num_listings,phone_num,agent_license,first_name,last_name
0,X1-ZUz193idtwb8qx_a9fjw,The Duncan Duo,/profile/TheDuncanDuo/,From our first meeting we knew we were in good...,Review 06/13/2022,https://photos.zillowstatic.com/h_g/ISznw4b133...,924 total reviews (302 local),4.9,(813) 514-8504,Agent License #: 3426012,18 Listings,302 Local Reviews,RE/MAX Dynamic,2022-06-13,2022,924.0,302.0,True,18,8135148504,3426012,The,Duo
5,X1-ZUxk5mqqkaqebt_7lwvs,The McIntosh Group ~ Jason McIntosh,/profile/jasonmcintoshremax/,He was very responsive during the entire proce...,Review 06/20/2022,https://photos.zillowstatic.com/h_g/IS7abp49n9...,255 total reviews (73 local),5.0,(813) 370-1494,Agent License #: SL3335722,2 Listings,73 Local Reviews,The McIntosh Group @ RE/MAX Realty Unlimited,2022-06-20,2022,255.0,73.0,True,2,8133701494,SL3335722,The,McIntosh
6,X1-ZUyh99meohlvrd_4srxs,"Archie W. Campbell, III",/profile/archiecampbell/,Since we had no idea what we were doing or wha...,Review 09/14/2021,https://photos.zillowstatic.com/h_g/ISzbbs5w7r...,71 total reviews (32 local),5.0,(813) 710-4608,Agent License #: 3033601,2 Listings,32 Local Reviews,RE/MAX Action First Of Florida,2021-09-14,2021,71.0,32.0,True,2,8137104608,3033601,Archie,III
7,X1-ZUympqnfpqmfbd_64j9u,Amy Smith Becky Kuntz,/profile/NextHomeBay-Area-Exp/,"We were out of state, so Amy did video tours a...",Review 08/30/2021,https://photos.zillowstatic.com/h_g/ISfcyifonn...,49 total reviews (25 local),5.0,(813) 537-6082,,2 Listings,25 Local Reviews,NextHome Bay Area Experts,2021-08-30,2021,49.0,25.0,True,2,8135376082,NaN,Amy,Kuntz
8,X1-ZUz07vdho8fvgp_59mrg,Kenny Hayslett,/profile/Kenny-Hayslett/,"The housing market is in Asante right now, but...",Review 06/30/2021,https://photos.zillowstatic.com/h_g/IS1e65786c...,103 total reviews (10 local),5.0,(727) 314-7983,Agent License #: 3075786,2 Listings,10 Local Reviews,RE/MAX Action First,2021-06-30,2021,103.0,10.0,True,2,7273147983,3075786,Kenny,Hayslett
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1,X1-ZUxuawsfr4in0p_5qhl4,Edward R. Marley IV,/profile/emarley89/,"He was patient, courteous and professional.",Review 07/15/2021,https://photos.zillowstatic.com/h_g/IS3r98yinz...,8 total reviews (2 local),5.0,(813) 758-1898,,No Listings,2 Local Reviews,Charles Rutenberg Realty,2021-07-15,2021,8.0,2.0,True,No Listings,8137581898,NaN,Edward,IV
2,X1-ZUzqqketnv6adl_1vfad,Ekaterine Aguilar PA,/profile/eaguilar5/,"We are extremely grateful for her efforts, and...",Review 05/13/2022,https://photos.zillowstatic.com/h_g/ISxfycwdhp...,6 total reviews (2 local),5.0,(352) 661-2485,Agent License #: 3281374,No Listings,2 Local Reviews,People's Trust Realty,2022-05-13,2022,6.0,2.0,True,No Listings,3526612485,3281374,Ekaterine,PA
3,X1-ZUwzs77t0wy6fd_56tmi,Erica Baur,/profile/ericabaur/,Every interaction was personable yet professio...,Review 02/09/2022,https://photos.zillowstatic.com/h_g/ISa953wocy...,4 total reviews (2 local),5.0,(352) 644-9627,Agent License #: 3381502,No Listings,2 Local Reviews,Florida Tropics Realty,2022-02-09,2022,4.0,2.0,True,No Listings,3526449627,3381502,Erica,Baur
5,X1-ZU10l5qqagq0etl_68qz9,Gary and Mary Jo Davis,/profile/Gary-Davis16/,Their knowledge and resources made me much mor...,Review 03/09/2021,https://photos.zillowstatic.com/h_g/ISyfmzpawk...,31 total reviews (2 local),5.0,(813) 846-6390,Agent License #: 3349588,No Listings,2 Local Reviews,BHHS Florida Properties - The Husky Team,2021-03-09,2021,31.0,2.0,True,No Listings,8138466390,3349588,Gary,Davis


Step #5 - Export file

In [ ]:
# uncomment below to download file
df_filter.to_csv('agent_contact_data.csv', index=False)
files.download('agent_contact_data.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# End Notebook